In [112]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import json
from dataloader import Twitch
import tensorflow as tf
from spektral.layers import GCNConv
from tensorflow.keras import layers
from utils import *
from spektral.data.loaders import SingleLoader
from sklearn.model_selection import train_test_split
from spektral.layers import GCNConv
from spektral.models.gcn import GCN
from spektral.transforms import LayerPreprocess
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.callbacks import EarlyStopping


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
def build_FFN(hidden_units, dropout_rate, name=None):
    fnn_layers = []

    for units in hidden_units:
        fnn_layers.append(layers.BatchNormalization())
        fnn_layers.append(layers.Dropout(dropout_rate))
        fnn_layers.append(layers.Dense(units, activation=tf.nn.gelu))

    return keras.Sequential(fnn_layers, name=name)

In [78]:
class GNNNodeClassifier(tf.keras.Model):
    def __init__(self, num_classes, hidden_channels, out_channels, dropout_rate=0.2):
        self.conv1 = GCNConv(hidden_channels)
        self.conv2 = GCNConv(hidden_channels)
        #self.ffn = build_FFN(hidden_channels, dropout_rate)
        self.fn1 = layers.Dense(num_classes)
    
    def call(self, inputs):
        x, adj = inputs
        
        x = self.conv1([x, adj])
        x = self.conv2([x, adj])
        x = self.fn1(x)

        return x

In [102]:
def mask_to_weights(mask):
    return mask.astype(np.float32) / np.count_nonzero(mask)

In [126]:
dataset = Twitch("DE", normalize_x=True, transforms=[LayerPreprocess(GCNConv)])

weights_tr, weights_va, weights_te = (
    mask_to_weights(mask)
    for mask in (dataset.mask_tr, dataset.mask_va, dataset.mask_te)
)

In [127]:
loader_tr = SingleLoader(dataset, sample_weights=weights_tr)
loader_va = SingleLoader(dataset, sample_weights=weights_va)

In [128]:
model = GCN(n_labels=dataset.n_labels, n_input_channels=dataset.n_node_features)


In [129]:
model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss=BinaryCrossentropy(),
    weighted_metrics=["acc"],
)

In [131]:
lr = 1e-2
epochs = 200
patience = 10

model.fit(
    loader_tr.load(),
    steps_per_epoch=loader_va.steps_per_epoch,
    validation_data=loader_va.load(),
    validation_steps=loader_va.steps_per_epoch,
    epochs=epochs,
    callbacks=[EarlyStopping(patience=patience, restore_best_weights=True)],
)

Epoch 1/200
1/1 [==============================] - 0s 212ms/step - loss: 0.0041 - acc: 0.6046 - val_loss: 0.0038 - val_acc: 0.6042
Epoch 2/200
1/1 [==============================] - 0s 211ms/step - loss: 0.0038 - acc: 0.6046 - val_loss: 0.0035 - val_acc: 0.6042
Epoch 3/200
1/1 [==============================] - 0s 210ms/step - loss: 0.0035 - acc: 0.6046 - val_loss: 0.0032 - val_acc: 0.6042
Epoch 4/200
1/1 [==============================] - 0s 212ms/step - loss: 0.0032 - acc: 0.6046 - val_loss: 0.0030 - val_acc: 0.6042
Epoch 5/200
1/1 [==============================] - 0s 210ms/step - loss: 0.0030 - acc: 0.6046 - val_loss: 0.0027 - val_acc: 0.6042
Epoch 6/200
1/1 [==============================] - 0s 208ms/step - loss: 0.0027 - acc: 0.6046 - val_loss: 0.0025 - val_acc: 0.6042
Epoch 7/200
1/1 [==============================] - 0s 210ms/step - loss: 0.0025 - acc: 0.6046 - val_loss: 0.0023 - val_acc: 0.6042
Epoch 8/200
1/1 [==============================] - 0s 210ms/step - loss: 0.0023 - a

KeyboardInterrupt: 

In [132]:
print("Evaluating model.")
loader_te = SingleLoader(dataset, sample_weights=weights_te)
eval_results = model.evaluate(loader_te.load(), steps=loader_te.steps_per_epoch)
print("Done.\n" "Test loss: {}\n" "Test accuracy: {}".format(*eval_results))

Evaluating model.
1/1 [==============================] - 0s 154ms/step - loss: 7.2833e-05 - acc: 0.6047
Done.
Test loss: 7.283292507054284e-05
Test accuracy: 0.604736864566803
